In [49]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from repeater import retry
import json


In [50]:
icon_service = IconService(HTTPProvider("https://sejong.net.solidwallet.io", 3))
NID = 83
NONCE = 100
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

# checking transaction result
def check_tx_result(res):
    if res['status'] == 0:
        print("Tx Failed, Message: " + res['failure']['message'])
    else:
        print("Success")
        
        
GOVERNANCE_SCORE = "cx06eaddea58321a98bd0e1c9bd70f4680d4f3a73a"
DEX_SCORE = "cx113466143604608cbf58d84674d9360b29f08530"
LOANS_SCORE = "cx2148bdb92054b2306076f3579c0ab7fced91a9ca"


#MAINNET ADDRESS
# GOVERNANCE_SCORE = "cx44250a12074799e26fdeee75648ae47e2cc84219"
# DEX_SCORE = "cx481f7e10c7c4fc59d6ac68472018f10977a91b39"
#LOANS_SCORE = "cx66d4d90f5f113eba575bf793570135f9b10cece1"



In [51]:
with open("./keystore/staking_test.pwd", "r") as f:
    key_data = f.read()
deployer_wallet = KeyWallet.load("./keystore/staking_test.json", key_data)

#using private key to load test wallet to perform transactions in case of need
# tester_wallet = KeyWallet.load(bytes.fromhex(""))
# tester_wallet.get_address()

In [52]:
def deploy_contract(wallet, _to_address, _name, params):
    deploy_transaction = DeployTransactionBuilder() \
        .from_(wallet) \
        .to(_to_address) \
        .nid(NID) \
        .params(params) \
        .version(3) \
        .nonce(100) \
        .content_type("application/zip") \
        .content(gen_deploy_data_content(_name)) \
        .build()

    estimate_step = icon_service.estimate_step(deploy_transaction)
    step_limit = estimate_step + 40000000
    signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)

    return icon_service.send_transaction(signed_transaction)


def send_tx(wallet, _to_address, method, params, value):
    transaction = CallTransactionBuilder() \
        .from_(wallet.get_address()) \
        .to(_to_address) \
        .value(value) \
        .step_limit(10000000) \
        .nid(NID) \
        .nonce(100) \
        .method(method) \
        .params(params) \
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)

    return get_tx_result(tx_hash)


def call_tx(_to_address, method, params):
    call = CallBuilder() \
        .to(_to_address) \
        .method(method) \
        .params(params) \
        .build()

    return icon_service.call(call)

#swap function for DEX
def swap(from_token, to_token, amount,wallet):
    data2 = json.dumps({"method": "_swap", "params": {"toToken":str(to_token)}})
    params = {'_to': contract_addresses['dex'], '_value': amount, '_data': data2.encode()}
    transaction = CallTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(from_token)\
        .step_limit(10000000)\
        .value(0)\
        .nid(NID)\
        .nonce(100)\
        .method("transfer")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    return get_tx_result(tx_hash)

In [53]:
# Getting all the SCORE address from GOVERNANCE SCORE
# The result shouldn't have router and feehandler in keys
contract_addresses = call_tx(GOVERNANCE_SCORE, 'getAddresses', {})
print(contract_addresses)

{'baln': 'cx44b142082a0612f60a5cff9e4204773b4799b102', 'bnUSD': 'cx50b5bc7d1088848ff106601d6a194aa16e6f9d83', 'bwt': 'cx655079c90d80b378819eddf3b05501392ecb0074', 'daofund': 'cx38e81d7b705784b2ddb586f0fdd3616e565b18b3', 'dex': 'cx113466143604608cbf58d84674d9360b29f08530', 'dividends': 'cxaab78dec61aded71309e9529feaf25e740526c63', 'feehandler': 'cx941a4bcca5c25d249e4ea7bef05edce854eabe40', 'loans': 'cx2148bdb92054b2306076f3579c0ab7fced91a9ca', 'oracle': 'cx0999df3af49a227fc8799f0d78d19cd8fe4bb76d', 'rebalancing': 'cxc2d1b9bbdfa51cf404c47e229c71773954acc453', 'reserve': 'cx513777210d38e8c3ba4a5f6ecd313f7bda86c4f2', 'rewards': 'cx4fb4db59d14adc6b9f6ed3d31f1154415bbe4e7e', 'router': 'cx7e58ab69fd019eaf56fd072d00f1634b64f80d13', 'sicx': 'cx0508faa2f6b1f333fa84190db7c80c9dbf822cb4', 'staking': 'cx6b88083a463739c94f1aa76341df1a21bddca2be'}


In [69]:
# Taking loans and swapping tokens on dex 

print("Taking loans")
res = send_tx(deployer_wallet, contract_addresses['loans'], "depositAndBorrow", {'_asset': 'bnUSD', '_amount': 11 * 10 **18}, 50 * 10 **18)
check_tx_result(res)


res = swap(contract_addresses['bnUSD'], contract_addresses['sicx'],5*10**18, deployer_wallet)
check_tx_result(res)
#print txHash incase of need
# print(res['txHash'])

# res = swap(contract_addresses['sicx'], contract_addresses['bnUSD'],15*10**18, deployer_wallet)
# check_tx_result(res)

# res = swap(data['bnUSD'], usds_score,50*10**18, deployer_wallet)
# check_tx_result(res)

# res = swap(data['bnUSD'], data['sicx'],50*10**18, deployer_wallet)
# check_tx_result(res)

# res = swap(data['sicx'], omm_score,10*10**18, deployer_wallet)
# check_tx_result(res)

# res = swap(omm_score, data['sicx'],5*10**18, deployer_wallet)
# check_tx_result(res)


Taking loans
cx2148bdb92054b2306076f3579c0ab7fced91a9ca
cx50b5bc7d1088848ff106601d6a194aa16e6f9d83
cx0508faa2f6b1f333fa84190db7c80c9dbf822cb4
Retrying in 1 seconds...
Retrying in 2 seconds...
Success
Retrying in 1 seconds...
Retrying in 2 seconds...
Success


# PHASE 1

In Phase 1 , 
We deploy:
    Router contract,
    Fee handler contract and
    Governance Contract (update)

## DEPLOY ROUTER CONTRACT

In [57]:
# We need to set all the staking sicx and dex address on router on_install 
res = get_tx_result(deploy_contract(deployer_wallet.get_address(), GOVERNANCE_ADDRESS, './core_contracts/router',
                                   {"_governance": GOVERNANCE_SCORE}))
print('scoreAddress:', res.get('scoreAddress', ''))
router_score = res.get('scoreAddress', '')
check_tx_result(res)

Retrying in 1 seconds...
Retrying in 2 seconds...
scoreAddress: cx2774afbaaf39fa395024fdb8d2a10dd9823c8490
Success


## UPDATE GOVERNANCE CONTRACT

In [58]:
tx = get_tx_result(
    deploy_contract(deployer_wallet.get_address(), GOVERNANCE_SCORE, './core_contracts/governance', {}))
print('scoreAddress:', tx.get('scoreAddress', ''))
check_tx_result(tx)

# In this readonly-call , the feehandler and router score should show None.
print(call_tx(GOVERNANCE_SCORE, 'getAddresses', {}))


Retrying in 1 seconds...
scoreAddress: cx06eaddea58321a98bd0e1c9bd70f4680d4f3a73a
Success
{'baln': 'cx44b142082a0612f60a5cff9e4204773b4799b102', 'bnUSD': 'cx50b5bc7d1088848ff106601d6a194aa16e6f9d83', 'bwt': 'cx655079c90d80b378819eddf3b05501392ecb0074', 'daofund': 'cx38e81d7b705784b2ddb586f0fdd3616e565b18b3', 'dex': 'cx113466143604608cbf58d84674d9360b29f08530', 'dividends': 'cxaab78dec61aded71309e9529feaf25e740526c63', 'feehandler': 'cx941a4bcca5c25d249e4ea7bef05edce854eabe40', 'loans': 'cx2148bdb92054b2306076f3579c0ab7fced91a9ca', 'oracle': 'cx0999df3af49a227fc8799f0d78d19cd8fe4bb76d', 'rebalancing': 'cxc2d1b9bbdfa51cf404c47e229c71773954acc453', 'reserve': 'cx513777210d38e8c3ba4a5f6ecd313f7bda86c4f2', 'rewards': 'cx4fb4db59d14adc6b9f6ed3d31f1154415bbe4e7e', 'router': 'cx7e58ab69fd019eaf56fd072d00f1634b64f80d13', 'sicx': 'cx0508faa2f6b1f333fa84190db7c80c9dbf822cb4', 'staking': 'cx6b88083a463739c94f1aa76341df1a21bddca2be'}


## DEPLOY FEE HANDLER

In [59]:
res = get_tx_result(deploy_contract(deployer_wallet.get_address(), GOVERNANCE_ADDRESS, './core_contracts/fees',
                                   {"_governance": GOVERNANCE_SCORE}))
print('scoreAddress:', res.get('scoreAddress', ''))
FEEHANDLER_SCORE = res.get('scoreAddress', '')
check_tx_result(res)

Retrying in 1 seconds...
Retrying in 2 seconds...
scoreAddress: cx4131b1b29bd66f162a4d7537fdc20f6aa3ae9782
Success


# PHASE 2 (Once the contract is accepted)

We set the fee handler and router addresses in governance contract and also sets all the function of fee handler.

#### SET ROUTER ADDRESS AND FEEHANDLER ADDRESS IN GOVERNANCE

In [60]:
contract_addresses['router'] = router_score
contract_addresses['feehandler'] = FEEHANDLER_SCORE
res = send_tx(deployer_wallet, GOVERNANCE_SCORE, "setAddresses", {"_addresses": contract_addresses}, 0)
check_tx_result(res)

# the read-only fn should return the fee handler and router address instead of None.
print(call_tx(GOVERNANCE_SCORE, 'getAddresses', {}))

Retrying in 1 seconds...
Success
{'baln': 'cx44b142082a0612f60a5cff9e4204773b4799b102', 'bnUSD': 'cx50b5bc7d1088848ff106601d6a194aa16e6f9d83', 'bwt': 'cx655079c90d80b378819eddf3b05501392ecb0074', 'daofund': 'cx38e81d7b705784b2ddb586f0fdd3616e565b18b3', 'dex': 'cx113466143604608cbf58d84674d9360b29f08530', 'dividends': 'cxaab78dec61aded71309e9529feaf25e740526c63', 'feehandler': 'cx4131b1b29bd66f162a4d7537fdc20f6aa3ae9782', 'loans': 'cx2148bdb92054b2306076f3579c0ab7fced91a9ca', 'oracle': 'cx0999df3af49a227fc8799f0d78d19cd8fe4bb76d', 'rebalancing': 'cxc2d1b9bbdfa51cf404c47e229c71773954acc453', 'reserve': 'cx513777210d38e8c3ba4a5f6ecd313f7bda86c4f2', 'rewards': 'cx4fb4db59d14adc6b9f6ed3d31f1154415bbe4e7e', 'router': 'cx2774afbaaf39fa395024fdb8d2a10dd9823c8490', 'sicx': 'cx0508faa2f6b1f333fa84190db7c80c9dbf822cb4', 'staking': 'cx6b88083a463739c94f1aa76341df1a21bddca2be'}


#### SET ACCEPTED DIVIDENDS TOKEN

In [61]:
# call the tx through Governance SCORE.

res2 = send_tx(deployer_wallet, GOVERNANCE_SCORE, "setAcceptedDividendTokens", {'_tokens': [contract_addresses['sicx'],
            contract_addresses['bnUSD'],
            contract_addresses["baln"]]}, 0)
check_tx_result(res2)
call_tx(FEEHANDLER_SCORE, 'getAcceptedDividendTokens', {})

Retrying in 1 seconds...
Retrying in 2 seconds...
Success


['cx0508faa2f6b1f333fa84190db7c80c9dbf822cb4',
 'cx50b5bc7d1088848ff106601d6a194aa16e6f9d83',
 'cx44b142082a0612f60a5cff9e4204773b4799b102']

#### SET FEE PROCESSING INTERVAL

In [62]:
res = send_tx(deployer_wallet, GOVERNANCE_SCORE, "setFeeProcessingInterval", {"_interval": 100}, 0)
check_tx_result(res)
call_tx(FEEHANDLER_SCORE, 'getFeeProcessingInterval', {})

Retrying in 1 seconds...
Retrying in 2 seconds...
Success


'0x64'

#### SET ROUTE

In [63]:
toToken = contract_addresses['baln']     
#sejong_address
omm_score = "cx683c78174c740c3c6215de0c3ead1fad024324d6"
usds_score ="cx2e008873de78f5bf82008a46b529d94924d9b6ae" 
iusdc_score = "cxb2c075e9130440dd64e6bfd3fe09e5c629f6e183"

#mainnet address
# usdc_score = "cxbb2871f468a3008f80b08fdde5b8b951583acf06"
# iusdc_score = "cxae3034235540b924dfcc1b45836c293dcc82bfb7"
# omm_score = "cx1a29259a59f463a67bb2ef84398b30ca56b5830a"
# craft_score = "cx2e6d0fc0eca04965d06038c8406093337f085fcf"
# metanyx_score = "cx369a5f4ce4f4648dfc96ba0c8229be0693b4eca2"
# iusdt_score = "cx3a36ea1f6b9aa3d2dd9cb68e8987bcc3aabaaa88"

# FOR OMM
fromToken = omm_score
path = [ contract_addresses['sicx'],
        contract_addresses['baln']
        ]
res = send_tx(deployer_wallet, GOVERNANCE_SCORE, "setRoute",  {'_fromToken':fromToken, '_toToken':toToken, '_path': path}, 0)
check_tx_result(res)
print(call_tx(FEEHANDLER_SCORE, 'getRoute', {"_fromToken":fromToken,"_toToken":contract_addresses['baln']}))

# FOR usds
# fromToken = usds_score
# path = [ contract_addresses['bnUSD'],
#         contract_addresses['baln']
#         ]
# res = send_tx(deployer_wallet, GOVERNANCE_SCORE, "setRoute",  {'_fromToken':fromToken, '_toToken':toToken, '_path': path}, 0)
# check_tx_result(res)
# print(call_tx(FEEHANDLER_SCORE, 'getRoute', {"_fromToken":fromToken,"_toToken":contract_addresses['baln']}))

# FOR iusdc
# fromToken = iusdc_score
# path = [ contract_addresses['bnUSD'],
#         contract_addresses['baln']
#         ]
# res = send_tx(deployer_wallet, GOVERNANCE_SCORE, "setRoute",  {'_fromToken':fromToken, '_toToken':toToken, '_path': path}, 0)
# check_tx_result(res)
# print(call_tx(FEEHANDLER_SCORE, 'getRoute', {"_fromToken":fromToken,"_toToken":contract_addresses['baln']}))

# FOR craft
# fromToken = craft_score
# path = [ data['sicx'],
#         data['baln']
#         ]
# send_tx(deployer_wallet, FEEHANDLER_SCORE, "setRoute",  {'_fromToken':fromToken, '_toToken':toToken, '_path': path}, 0)

# FOR metanyx
# fromToken = metanyx_score
# path = [ data['bnUSD'],
#         data['baln']
#         ]
# send_tx(deployer_wallet, FEEHANDLER_SCORE, "setRoute",  {'_fromToken':fromToken, '_toToken':toToken, '_path': path}, 0)

# FOR iusdt
# fromToken = iusdt_score
# path = [ data['bnUSD'],
#         data['baln']
#         ]
# send_tx(deployer_wallet, FEEHANDLER_SCORE, "setRoute",  {'_fromToken':fromToken, '_toToken':toToken, '_path': path}, 0)


Retrying in 1 seconds...
Retrying in 2 seconds...
Success
{'fromToken': 'cx683c78174c740c3c6215de0c3ead1fad024324d6', 'path': ['cx0508faa2f6b1f333fa84190db7c80c9dbf822cb4', 'cx44b142082a0612f60a5cff9e4204773b4799b102'], 'toToken': 'cx44b142082a0612f60a5cff9e4204773b4799b102'}


## UPDATE DEX CONTRACT

In [64]:
# We need to set fee handler address on on_update.
tx = get_tx_result(
    deploy_contract(deployer_wallet.get_address(), DEX_SCORE, './core_contracts/dex', {}))
print('scoreAddress:', tx.get('scoreAddress', ''))
check_tx_result(tx)

Retrying in 1 seconds...
Retrying in 2 seconds...
scoreAddress: cx113466143604608cbf58d84674d9360b29f08530
Success


## UPDATE LOANS CONTRACT

In [65]:
tx = get_tx_result(
    deploy_contract(deployer_wallet.get_address(), LOANS_SCORE, './core_contracts/loans', {}))
print('scoreAddress:', tx.get('scoreAddress', ''))
check_tx_result(tx)


Retrying in 1 seconds...
Retrying in 2 seconds...
scoreAddress: cx2148bdb92054b2306076f3579c0ab7fced91a9ca
Success


In [70]:
print("Taking loans")
res = send_tx(deployer_wallet, contract_addresses['loans'], "depositAndBorrow", {'_asset': 'bnUSD', '_amount': 100 * 10 **18}, 500 * 10 **18)
check_tx_result(res)

#Swapping bnUSD to sICX
res = swap(contract_addresses['bnUSD'], contract_addresses['sicx'],5*10**18,deployer_wallet)
check_tx_result(res)

Taking loans
cx2148bdb92054b2306076f3579c0ab7fced91a9ca
cx50b5bc7d1088848ff106601d6a194aa16e6f9d83
cx0508faa2f6b1f333fa84190db7c80c9dbf822cb4
Retrying in 1 seconds...
Retrying in 2 seconds...
Retrying in 4 seconds...
Retrying in 8 seconds...
Retrying in 16 seconds...
Retrying in 32 seconds...


KeyboardInterrupt: 